<div style="background-color: #f0f8ff; border-left: 6px solid #2196F3; padding: 15px; border-radius: 5px;">

# 🚀 Analyse Exploratoire du Marché Immobilier Français (2021-2025)

**Objectif :** Ce notebook effectue une analyse exploratoire des données de Demandes de Valeurs Foncières (DVF) sur 5 ans. L'objectif est de fournir à notre persona, **Alexandre Dubois**, une première visualisation des grandes tendances du marché pour guider ses futurs investissements.

**Méthodologie (Optimisée) :**
* **1. Configuration :** Définition des constantes et des schémas de données.
* **2. Chargement Optimisé (Parquet) :** Nous utilisons un DataFrame "Macro" (`df_clean`) pour toutes les analyses nationales et départementales. Il est chargé instantanément depuis un fichier Parquet.
* **3. Analyse "Macro" (Besoins I, II, III.1, III.2) :** Analyse des tendances, carte interactive et stratégie des départements. Tout est fluide car basé sur `df_clean`.
* **4. Analyse "Micro" (Besoin III.3) :** Pour l'analyse des villes, nous effectuons un **chargement chirurgical** (on-demand) des CSV pour ne récupérer *que* les données des 5 villes cibles. Cela préserve notre RAM principale.
* **5. Outils d'Export :** Fonctions pour exporter les analyses.
</div>

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
import base64
import os
import gc
import io

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)
px.defaults.template = "plotly_white"

# --- CHEMIN DU FICHIER PARQUET (Doit correspondre au 1er notebook) ---
PARQUET_FILE_PATH = 'dvf_clean_2021-2025.parquet'
df_clean = pd.DataFrame() # Initialisation
if os.path.exists(PARQUET_FILE_PATH):
    print(f"✅ Chargement rapide depuis '{PARQUET_FILE_PATH}'...")
    df_clean = pd.read_parquet(PARQUET_FILE_PATH)
    print(f"   -> Chargement terminé ({len(df_clean):,} lignes).")
    print("\n--- Informations et Utilisation Mémoire (Macro - Optimisée) ---")
    df_clean.info(memory_usage='deep')
else:
    print(f"❌ ERREUR: Le fichier Parquet '{PARQUET_FILE_PATH}' n'a pas été trouvé.")
    print("   Veuillez d'abord exécuter le notebook 'visualisation.ipynb'.")

# Nettoyage initial de la RAM
gc.collect()

<div style='background-color: #fffbea; border-left: 6px solid #ffeb3b; padding: 15px; border-radius: 5px;'>

## 💡 Analyse des Prix et Tendances

**Fonction :** `analyze_price_by_type_size()`  
Cette étape éclaire les écarts de prix selon le type et la taille du bien.  
Pour Alexandre, elle répond au besoin d’**identifier les segments rentables** : appartements familiaux, petits studios à rendement rapide, ou maisons en périphérie à potentiel d’appréciation.  
L’objectif : replacer chaque donnée dans une vision claire du marché.

</div>


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# Assurez-vous que df_clean est chargé depuis la cellule précédente

def analyze_price_by_type_size(df):
    """
    Analyse le prix médian/m² et le volume par type de bien et nombre de pièces.
    CORRIGÉ pour utiliser go.Bar avec make_subplots.
    """
    if df.empty:
        print("❌ Pas de données.")
        return

    print("📊 Analyse du prix/m² par type et taille...")

    # Filtrer sur les types principaux et nombre de pièces raisonnable
    df_filtered = df[
        df['type_local'].isin(['Maison', 'Appartement']) &
        (df['nombre_pieces_principales'] >= 1) &
        (df['nombre_pieces_principales'] <= 6) # Limiter pour la clarté
    ].copy()

    # Convertir le nb de pièces en entier pour un affichage propre
    # Utiliser .astype('Int64') pour gérer les NaN potentiels avant conversion
    df_filtered['nombre_pieces_principales'] = df_filtered['nombre_pieces_principales'].astype('Int64')

    # Agréger les données
    df_agg = df_filtered.groupby(['type_local', 'nombre_pieces_principales']).agg(
        prix_median_m2=('prix_m2', 'median'),
        volume_transactions=('prix_m2', 'size') # 'size' est plus direct pour compter les lignes
    ).reset_index()

    # Filtrer les catégories avec peu de transactions (ex: moins de 1000) pour la clarté
    df_agg = df_agg[df_agg['volume_transactions'] > 1000].sort_values(by='nombre_pieces_principales') # Trier pour l'ordre des barres

    # --- Visualisation CORRIGÉE ---
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Prix Médian/m²", "Volume de Transactions"))

    # Couleurs distinctes pour Maison et Appartement
    colors = {'Maison': 'blue', 'Appartement': 'red'}
    legend_added = {'Prix Médian': set(), 'Volume': set()} # Pour éviter les légendes dupliquées

    for type_loc in ['Maison', 'Appartement']:
        df_type = df_agg[df_agg['type_local'] == type_loc]
        color = colors[type_loc]

        # --- Graphique Prix ---
        show_legend_prix = type_loc not in legend_added['Prix Médian']
        fig.add_trace(
            go.Bar(
                x=df_type['nombre_pieces_principales'],
                y=df_type['prix_median_m2'],
                name=type_loc, # Nom pour la légende
                marker_color=color, # Utiliser marker_color
                text=df_type['prix_median_m2'].round(0).astype(str) + ' €/m²',
                textposition='outside', # Mettre le texte à l'extérieur pour éviter chevauchement
                hovertemplate=f'<b>{type_loc} - %{{x}} pièces</b><br>Prix Médian: %{{y:,.0f}} €/m²<extra></extra>',
                showlegend=True, # Toujours afficher dans la légende principale
                legendgroup='prix', # Grouper les légendes
                offsetgroup=type_loc # Pour le mode groupé
            ), row=1, col=1
        )
        legend_added['Prix Médian'].add(type_loc)


        # --- Graphique Volume ---
        show_legend_vol = type_loc not in legend_added['Volume']
        fig.add_trace(
            go.Bar(
                x=df_type['nombre_pieces_principales'],
                y=df_type['volume_transactions'],
                name=type_loc, # Nom pour la légende
                marker_color=color, # Utiliser marker_color
                text=df_type['volume_transactions'].apply(lambda x: f'{x:,.0f}'),
                textposition='outside',
                hovertemplate=f'<b>{type_loc} - %{{x}} pièces</b><br>Volume: %{{y:,.0f}} transactions<extra></extra>',
                showlegend=False, # Masquer la légende pour le 2ème graph (déjà sur le 1er)
                legendgroup='volume', # Grouper les légendes
                offsetgroup=type_loc # Pour le mode groupé
            ), row=1, col=2
        )
        legend_added['Volume'].add(type_loc)


    fig.update_layout(
        title_text="<b>🔑 Prix et Volume par Type de Bien et Nombre de Pièces (2021-2025)</b>",
        title_x=0.5,
        barmode='group', # Mode groupé pour comparer Maison/Appartement côte à côte
        legend_title_text='Type de Bien',
        xaxis_title="Nombre de Pièces Principales",
        xaxis2_title="Nombre de Pièces Principales",
        yaxis_title="Prix Médian (€/m²)",
        yaxis2_title="Nombre de Transactions",
        height=500, # Ajuster la hauteur si besoin
        legend=dict(traceorder='reversed') # Pour avoir Maison puis Appartement dans la légende
    )
    # Ajuster les axes pour que le texte extérieur soit visible
    fig.update_yaxes(rangemode='tozero', row=1, col=1)
    fig.update_yaxes(rangemode='tozero', row=1, col=2)
    fig.update_traces(textfont_size=10, textangle=0, cliponaxis=False) # Améliorer lisibilité texte
    fig.show()

# --- Exécution ---
if 'df_clean' in locals() and not df_clean.empty:
    analyze_price_by_type_size(df_clean)
else:
    print("❌ Le DataFrame 'df_clean' n'est pas chargé. Exécutez la cellule précédente.")

<div style='background-color: #f0fff4; border-left: 6px solid #4CAF50; padding: 15px; border-radius: 5px;'>

## 📈 Analyse de la Saisonnalité

**Fonction :** `analyze_seasonality()`  
Cette fonction explore le rythme annuel du marché : mois dynamiques, creux saisonniers, périodes propices à l’achat ou à la vente.  
Pour un investisseur pressé et prudent, comprendre **quand** agir est aussi stratégique que savoir **où** placer son argent.

</div>


In [ ]:
def analyze_seasonality(df):
    """
    Analyse le volume de transactions et le prix médian/m² par mois/trimestre
    pour identifier une éventuelle saisonnalité (Besoin III.3).
    """
    if df.empty:
        print("❌ Pas de données pour l'analyse de saisonnalité.")
        return

    print("📅 Analyse de la saisonnalité du marché...")

    # Assurer que 'date_mutation' est datetime
    df['date_mutation'] = pd.to_datetime(df['date_mutation'], errors='coerce')
    df = df.dropna(subset=['date_mutation']) # Nettoyer si la conversion échoue

    # Extraire mois et trimestre si pas déjà présents (df_clean devrait les avoir via 'trimestre')
    if 'mois' not in df.columns:
        df['mois'] = df['date_mutation'].dt.month
    if 'trimestre' not in df.columns:
         df['trimestre'] = df['date_mutation'].dt.quarter

    # Convertir mois et trimestre en category pour l'ordre
    df['mois'] = pd.Categorical(df['mois'], categories=list(range(1, 13)), ordered=True)
    df['trimestre'] = pd.Categorical(df['trimestre'], categories=list(range(1, 5)), ordered=True)

    # Agréger par mois et par trimestre
    df_agg_mois = df.groupby('mois').agg(
        prix_median_m2=('prix_m2', 'median'),
        volume_transactions=('prix_m2', 'size')
    ).reset_index()

    df_agg_trim = df.groupby('trimestre').agg(
        prix_median_m2=('prix_m2', 'median'),
        volume_transactions=('prix_m2', 'size')
    ).reset_index()

    # --- Visualisation (2x2 subplots) ---
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            "Volume Mensuel Moyen (2021-2025)", "Prix Médian Mensuel (€/m²)",
            "Volume Trimestriel Moyen (2021-2025)", "Prix Médian Trimestriel (€/m²)"
        ),
        vertical_spacing=0.2
    )

    # Volume Mensuel
    fig.add_trace(go.Bar(x=df_agg_mois['mois'], y=df_agg_mois['volume_transactions'], name='Volume Mois', marker_color='skyblue'), row=1, col=1)
    # Prix Mensuel
    fig.add_trace(go.Scatter(x=df_agg_mois['mois'], y=df_agg_mois['prix_median_m2'], mode='lines+markers', name='Prix Mois', line=dict(color='orange')), row=1, col=2)

    # Volume Trimestriel
    fig.add_trace(go.Bar(x=df_agg_trim['trimestre'], y=df_agg_trim['volume_transactions'], name='Volume Trimestre', marker_color='lightcoral'), row=2, col=1)
    # Prix Trimestriel
    fig.add_trace(go.Scatter(x=df_agg_trim['trimestre'], y=df_agg_trim['prix_median_m2'], mode='lines+markers', name='Prix Trimestre', line=dict(color='green')), row=2, col=2)

    fig.update_layout(
        title_text="<b>🗓️ Saisonnalité du Marché Immobilier (2021-2025)</b>",
        title_x=0.5,
        height=700,
        showlegend=False
    )
    fig.update_xaxes(title_text="Mois", row=1, col=1)
    fig.update_xaxes(title_text="Mois", row=1, col=2)
    fig.update_xaxes(title_text="Trimestre", tickvals=[1, 2, 3, 4], ticktext=['T1', 'T2', 'T3', 'T4'], row=2, col=1)
    fig.update_xaxes(title_text="Trimestre", tickvals=[1, 2, 3, 4], ticktext=['T1', 'T2', 'T3', 'T4'], row=2, col=2)
    fig.update_yaxes(title_text="Nb. Transactions Moy.", row=1, col=1)
    fig.update_yaxes(title_text="Prix Médian (€/m²)", row=1, col=2)
    fig.update_yaxes(title_text="Nb. Transactions Moy.", row=2, col=1)
    fig.update_yaxes(title_text="Prix Médian (€/m²)", row=2, col=2)

    fig.show()

# --- Exécution ---
if 'df_clean' in locals() and not df_clean.empty:
    analyze_seasonality(df_clean.copy()) # Utiliser une copie pour éviter modif. inplace
else:
    print("❌ Le DataFrame 'df_clean' n'est pas chargé.")

<div style='background-color: #f9f0ff; border-left: 6px solid #9C27B0; padding: 15px; border-radius: 5px;'>

## 🏘️ Répartition des Types de Biens

**Fonction :** `analyze_property_type_distribution()`  
Cette analyse met en lumière la structure du marché : la part respective des maisons et des appartements dans les zones les plus actives.  
Alexandre y trouve un indicateur précieux pour évaluer la **demande locative dominante** et la **liquidité** des biens.

</div>


In [ ]:
import plotly.express as px

def analyze_property_type_distribution(df, top_n=20):
    """
    Visualise la proportion de Maisons vs. Appartements vendus
    dans les N départements les plus actifs (Besoin III.4).
    """
    if df.empty:
        print("❌ Pas de données pour l'analyse de répartition.")
        return

    print(f"🏘️ Analyse de la répartition Maisons/Appartements (Top {top_n} Départements)...")

    df_filtered = df[df['type_local'].isin(['Maison', 'Appartement'])].copy()

    # Compter les transactions par département et type
    df_counts = df_filtered.groupby(['code_departement', 'type_local']).size().unstack(fill_value=0)

    # Calculer le total et le pourcentage
    df_counts['Total'] = df_counts['Maison'] + df_counts['Appartement']
    # Éviter la division par zéro si un département n'a aucune transaction
    df_counts = df_counts[df_counts['Total'] > 0]
    df_counts['Pct_Maison'] = (df_counts['Maison'] / df_counts['Total']) * 100
    df_counts = df_counts.sort_values('Total', ascending=False).head(top_n).sort_values('Pct_Maison', ascending=False) # Trier par % Maison

    # --- Visualisation (Barres empilées 100%) ---
    fig = px.bar(
        df_counts,
        x=df_counts.index, # Code département
        # La deuxième colonne 'y' est calculée pour arriver à 100%
        y=['Pct_Maison', 100 - df_counts['Pct_Maison']],
        title=f"<b>📊 Répartition Maisons vs. Appartements (Top {top_n} Départements par Volume)</b>",
        labels={'x': 'Code Département', 'value': 'Pourcentage (%)'},
        hover_data={'Total': ':,d'}, # Afficher le volume total au survol
        height=500,
        # Laisser px.bar choisir les couleurs par défaut ou spécifier une map si besoin
        # color_discrete_map={'Pct_Maison': 'blue', 'wide_variable_1': 'red'} # Optionnel
    )

    # Mise à jour des noms pour la légende et le survol
    # Note: customdata[0] correspond à la première colonne ajoutée dans hover_data ('Total')
    fig.update_traces(hovertemplate='Département: %{x}<br>Pourcentage: %{y:.1f}%<br>Volume Total: %{customdata[0]:,d}<extra></extra>', selector=0)
    fig.update_traces(hovertemplate='Département: %{x}<br>Pourcentage: %{y:.1f}%<br>Volume Total: %{customdata[0]:,d}<extra></extra>', selector=1)

    # --- CORRECTION ICI ---
    # Le dictionnaire doit mapper le nom initial de la trace ('Pct_Maison') au nouveau nom
    newnames = {'Pct_Maison': '% Maison', 'wide_variable_1': '% Appartement'}
    fig.for_each_trace(lambda t: t.update(name = newnames[t.name]))
    # ---------------------

    fig.update_layout(
        yaxis_title="Pourcentage (%)",
        xaxis={'categoryorder':'array', 'categoryarray': df_counts.index.tolist()}, # Ordonner selon le tri Pct_Maison
        legend_title_text='Type de Bien',
        barmode='stack', # Empiler pour voir la proportion
        title_x=0.5
    )
    fig.show()

# --- Exécution ---
# Assurez-vous que df_clean est défini et non vide
if 'df_clean' in locals() and not df_clean.empty:
    analyze_property_type_distribution(df_clean.copy()) # Utiliser une copie par sécurité
else:
    print("❌ Le DataFrame 'df_clean' n'est pas chargé.")

<div style='background-color: #f4f4f4; border-left: 6px solid #607D8B; padding: 15px; border-radius: 5px;'>

## 🌍 Analyse du Potentiel de Croissance

**Fonction :** `analyze_growth_potential()`  
En croisant les prix médians avec leur évolution temporelle, cette fonction révèle les **zones émergentes** : celles où le marché est encore abordable mais en pleine montée.  
C’est la boussole d’Alexandre pour repérer les **villes à fort potentiel** avant qu’elles ne deviennent inaccessibles.

</div>


In [ ]:
import plotly.express as px
import pandas as pd # S'assurer que pandas est importé

def analyze_growth_potential(df):
    """
    Identifie les départements avec potentiel en croisant prix médian et évolution
    sur la période (Besoin III.1). CORRIGÉ pour gérer les noms de colonnes numériques.
    """
    if df.empty or 'annee' not in df.columns:
        print("❌ Pas de données ou colonne 'annee' manquante.")
        return

    print("📈 Analyse du potentiel de croissance par département...")

    # Années de début et de fin disponibles (converties en int pour comparaison)
    df['annee'] = pd.to_numeric(df['annee'], errors='coerce').dropna().astype(int)
    if df['annee'].nunique() < 2:
         print("❌ Nécessite au moins deux années de données pour calculer l'évolution.")
         return
    start_year = df['annee'].min()
    end_year = df['annee'].max()

    # Calculer le prix médian par département pour l'année de début et de fin
    median_prices = df[df['annee'].isin([start_year, end_year])].groupby(['code_departement', 'annee'])['prix_m2'].median().unstack()

    # Calculer le volume total pour la taille des points
    total_volume = df.groupby('code_departement').size().reset_index(name='Volume_Total')

    # Filtrer les départements présents aux deux dates et avec assez de volume
    median_prices = median_prices.dropna()
    analysis_data = pd.merge(median_prices, total_volume, on='code_departement')
    analysis_data = analysis_data[analysis_data['Volume_Total'] > 1000] # Seuil de volume

    if analysis_data.empty:
        print("❌ Pas assez de données ou de volume pour l'analyse de potentiel.")
        return

    # --- CORRECTION : Renommer les colonnes numériques en strings ---
    col_start_year_str = f'Prix_{start_year}'
    col_end_year_str = f'Prix_{end_year}'
    analysis_data = analysis_data.rename(columns={start_year: col_start_year_str, end_year: col_end_year_str})
    # -----------------------------------------------------------------

    # Calculer l'évolution en % et le prix récent en utilisant les NOUVEAUX noms
    analysis_data['Evolution_Pct'] = ((analysis_data[col_end_year_str] - analysis_data[col_start_year_str]) / analysis_data[col_start_year_str]) * 100
    analysis_data['Prix_Median_Recent'] = analysis_data[col_end_year_str]

    # --- Visualisation (Scatter Plot) ---
    fig = px.scatter(
        analysis_data,
        x='Prix_Median_Recent',
        y='Evolution_Pct',
        size='Volume_Total',
        color='Prix_Median_Recent', # Colorer par niveau de prix
        color_continuous_scale='RdYlBu_r',
        hover_name=analysis_data.index, # Code département
        hover_data={
            'Prix_Median_Recent': ':.0f €/m²',
            'Evolution_Pct': ':.1f%',
            'Volume_Total': ':,d',
            # --- CORRECTION : Utiliser les NOUVEAUX noms de colonnes ---
            col_start_year_str: ':.0f €/m²', # Afficher prix début
            col_end_year_str: ':.0f €/m²'   # Afficher prix fin
            # -------------------------------------------------------------
        },
        title=f"<b>💡 Potentiel de Croissance: Prix Récent vs. Évolution {start_year}-{end_year} (par Département)</b>",
        labels={
            'Prix_Median_Recent': f'Prix Médian {end_year} (€/m²)',
            'Evolution_Pct': f'Évolution {start_year}-{end_year} (%)',
            # --- CORRECTION : Mettre à jour les labels pour hover_data ---
            col_start_year_str: f'Prix {start_year} (€/m²)',
            col_end_year_str: f'Prix {end_year} (€/m²)'
            # --------------------------------------------------------------
        },
        size_max=40,
        height=600
    )

    # Ajouter des lignes pour délimiter les quadrants (ex: médiane des prix et évolution 0%)
    median_price_overall = analysis_data['Prix_Median_Recent'].median()
    fig.add_hline(y=0, line_dash="dash", line_color="grey")
    fig.add_vline(x=median_price_overall, line_dash="dash", line_color="grey")

    # Ajouter des annotations pour les quadrants (Optionnel)
    fig.add_annotation(x=median_price_overall*0.5, y=analysis_data['Evolution_Pct'].max()*0.8, text="Potentiel (Bas Prix, Haute Croiss.)", showarrow=False, font=dict(color='green'))
    fig.add_annotation(x=median_price_overall*1.5, y=analysis_data['Evolution_Pct'].min()*0.8, text="Risque (Haut Prix, Faible Croiss.)", showarrow=False, font=dict(color='red'))

    fig.update_layout(title_x=0.5, coloraxis_colorbar_title=f'Prix {end_year} (€/m²)')
    fig.show()

# --- Exécution ---
if 'df_clean' in locals() and not df_clean.empty:
    analyze_growth_potential(df_clean.copy()) # Utiliser une copie pour éviter modif. inplace
else:
    print("❌ Le DataFrame 'df_clean' n'est pas chargé.")

<div style='background-color: #fff8e1; border-left: 6px solid #ff9800; padding: 15px; border-radius: 5px;'>

## 💰 Distribution des Budgets — Vue par Département

**Fonction :** `analyze_budget_distribution(df, top_n=10)`  
Cette cellule examine la **distribution des valeurs foncières** (prix de vente total) pour les *N* départements les plus actifs. Elle utilise un diagramme en boîte (box plot) pour rendre visibles les médianes, l'étendue et les éventuels outliers, tout en limitant l’affichage aux ventes < 2 000 000 € pour garder le visuel lisible. Parce que oui, un seul manoir peut gâcher toute la lecture.

### Ce que ça apporte à Alexandre Dubois
- Permet d’identifier rapidement les départements où les transactions se concentrent et la dispersion des prix.  
- Aide à repérer les marchés **stables** (petite dispersion) vs **volatils** (grande dispersion) — utile pour évaluer la sécurité d’un investissement locatif.  
- Donne une première idée des budgets nécessaires par département, pratique pour filtrer les opportunités selon capacité d’emprunt.
</div>


In [ ]:
import plotly.express as px

def analyze_budget_distribution(df, top_n=10):
    """
    Montre la distribution des valeurs foncières (prix de vente total)
    pour les N départements les plus actifs (Besoin II.5).
    """
    if df.empty or 'valeur_fonciere' not in df.columns:
        print("❌ Pas de données ou colonne 'valeur_fonciere' manquante.")
        return

    print(f"💰 Analyse de la distribution des budgets (Top {top_n} Départements)...")

    # Identifier les top N départements par volume
    top_departments = df['code_departement'].value_counts().head(top_n).index.tolist()

    df_filtered = df[df['code_departement'].isin(top_departments)].copy()

    # Limiter la valeur foncière pour une meilleure visualisation (ex: < 2M€)
    df_filtered = df_filtered[df_filtered['valeur_fonciere'] < 2_000_000]

    # --- Visualisation (Box Plot) ---
    fig = px.box(
        df_filtered,
        x='code_departement',
        y='valeur_fonciere',
        color='code_departement', # Colorer par département
        title=f"<b>Distribution des Prix de Vente (Valeur Foncière < 2M€) - Top {top_n} Départements</b>",
        labels={
            'code_departement': 'Code Département',
            'valeur_fonciere': 'Valeur Foncière (€)'
        },
        points=False, # Masquer les points individuels pour la clarté
        category_orders={'code_departement': top_departments}, # Ordonner par volume
        height=500
    )

    fig.update_layout(title_x=0.5, showlegend=False)
    fig.update_yaxes(tickformat=",.0f") # Formatage des euros

    fig.show()

# --- Exécution ---
if 'df_clean' in locals() and not df_clean.empty:
    analyze_budget_distribution(df_clean)
else:
    print("❌ Le DataFrame 'df_clean' n'est pas chargé.")

In [ ]:
def get_growth_potential_data(df):
    """
    Fonction utilitaire (reprise de l'analyse 7) pour extraire les données
    de potentiel de croissance sans afficher le graphique.
    """
    if df.empty or 'annee' not in df.columns:
        return pd.DataFrame(), 0, 0, 0

    # Assurer que 'annee' est numérique
    df_copy = df.copy()
    df_copy['annee'] = pd.to_numeric(df_copy['annee'], errors='coerce').dropna().astype(int)
    if df_copy['annee'].nunique() < 2:
         return pd.DataFrame(), 0, 0, 0
    
    start_year = df_copy['annee'].min()
    end_year = df_copy['annee'].max()

    median_prices = df_copy[df_copy['annee'].isin([start_year, end_year])].groupby(['code_departement', 'annee'])['prix_m2'].median().unstack()
    total_volume = df_copy.groupby('code_departement').size().reset_index(name='Volume_Total')
    
    median_prices = median_prices.dropna()
    analysis_data = pd.merge(median_prices, total_volume, on='code_departement')
    analysis_data = analysis_data[analysis_data['Volume_Total'] > 1000]

    if analysis_data.empty:
        return pd.DataFrame(), 0, 0, 0

    col_start_year_str = f'Prix_{start_year}'
    col_end_year_str = f'Prix_{end_year}'
    analysis_data = analysis_data.rename(columns={start_year: col_start_year_str, end_year: col_end_year_str})

    analysis_data['Evolution_Pct'] = ((analysis_data[col_end_year_str] - analysis_data[col_start_year_str]) / analysis_data[col_start_year_str]) * 100
    analysis_data['Prix_Median_Recent'] = analysis_data[col_end_year_str]
    
    median_price_overall = analysis_data['Prix_Median_Recent'].median()
    
    # Renommer l'index pour qu'il devienne une colonne claire
    analysis_data.index.name = 'Code_Departement'
    analysis_data.reset_index(inplace=True)

    return analysis_data, median_price_overall, start_year, end_year

def generate_and_download_insights(df):
    """
    Génère le rapport d'insights final et fournit des liens de téléchargement.
    """
    print("🔄 Génération du rapport d'insights stratégiques...")
    
    analysis_data, median_price_overall, start_year, end_year = get_growth_potential_data(df)
    
    if analysis_data.empty:
        print("❌ Données insuffisantes pour générer le rapport (moins de 2 ans de données ou volume faible).")
        return

    # --- 1. Création des segments d'insights ---
    
    # Segment 1: Potentiel (Bas Prix, Haute Croissance)
    df_top_potential = analysis_data[
        (analysis_data['Prix_Median_Recent'] < median_price_overall) &
        (analysis_data['Evolution_Pct'] > 0)
    ].copy()
    df_top_potential['Segment'] = f'Potentiel (Prix < {median_price_overall:,.0f}€, Croissance +)'
    df_top_potential = df_top_potential.sort_values('Evolution_Pct', ascending=False)

    # Segment 2: Marchés Porteurs (Haut Prix, Haute Croissance)
    df_hot_markets = analysis_data[
        (analysis_data['Prix_Median_Recent'] >= median_price_overall) &
        (analysis_data['Evolution_Pct'] > 0)
    ].copy()
    df_hot_markets['Segment'] = f'Porteur (Prix > {median_price_overall:,.0f}€, Croissance +)'
    df_hot_markets = df_hot_markets.sort_values('Evolution_Pct', ascending=False)

    # Combiner les rapports
    df_report = pd.concat([df_top_potential, df_hot_markets])

    # Sélectionner et renommer les colonnes pour un rapport propre
    report_cols = [
        'Segment',
        'Code_Departement',
        'Prix_Median_Recent',
        'Evolution_Pct',
        'Volume_Total',
        f'Prix_{start_year}',
        f'Prix_{end_year}'
    ]
    df_report = df_report[report_cols].reset_index(drop=True)
    
    # Arrondir pour la lisibilité
    df_report['Prix_Median_Recent'] = df_report['Prix_Median_Recent'].round(0)
    df_report['Evolution_Pct'] = df_report['Evolution_Pct'].round(2)
    df_report[f'Prix_{start_year}'] = df_report[f'Prix_{start_year}'].round(0)
    df_report[f'Prix_{end_year}'] = df_report[f'Prix_{end_year}'].round(0)

    print("✅ Rapport d'insights final généré :")
    display(df_report.head(10))

    # --- 2. Création des liens de téléchargement ---
    
    # Fonction pour créer le lien de téléchargement
    def create_download_link(df, title, filename, file_format):
        if file_format == 'excel':
            output = io.BytesIO()
            with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
                df.to_excel(writer, sheet_name='Report_Insights', index=False)
            data = output.getvalue()
            mime_type = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
        elif file_format == 'csv':
            data = df.to_csv(index=False, encoding='utf-8-sig') # utf-8-sig pour Excel
            data = data.encode('utf-8')
            mime_type = 'text/csv'
        
        b64 = base64.b64encode(data).decode()
        link = f'<a href="data:{mime_type};base64,{b64}" download="{filename}">{title}</a>'
        return link

    # Générer les liens
    excel_link = create_download_link(df_report, "📥 Télécharger le Rapport (.xlsx)", "DVF_Insights_Rapport.xlsx", "excel")
    csv_link = create_download_link(df_report, "📥 Télécharger le Rapport (.csv)", "DVF_Insights_Rapport.csv", "csv")

    display(HTML(f"<h3>Exports du Rapport :</h3> {excel_link} &nbsp;&nbsp;|&nbsp;&nbsp; {csv_link}"))

# --- Exécution ---
if 'df_clean' in locals() and not df_clean.empty:
    generate_and_download_insights(df_clean)
else:
    print("❌ Le DataFrame 'df_clean' n'est pas chargé. Impossible de générer le rapport.")